# 1. Giới thiệu

Home credit là một cuộc thi phân tích dữ liệu do kaggle tổ chức. Đề tài cần giải quyết là một bài toán thuộc lớp mô hình phân loại và học có giám sát. Yêu cầu của các đội thi là xây dựng mô hình phân loại các hợp đồng tín dụng đã được gán nhãn repaid (trả nợ) và no repaid  (không trả nợ) sao cho mức độ chính xác trong phân loại là lớn nhất. Tiêu chí được sử dụng để đánh giá mô hình là chỉ số AUC, một chỉ số khá quen thuộc trong lĩnh vực scorecard. Về mặt toán học AUC chính là phần diện tích nằm dưới đường cong ROC mà giá trị của nó nằm trong khoảng từ [0, 1]. Một mô hình càng có sức mạnh phân loại tốt khi AUC càng gần 1 và trái lại. Trường hợp AUC = 0.5 kết quả mô hình bằng với việc dự báo ngẫu nhiên. AUC < 0.5 đảo ngược kết quả của mô hình bạn sẽ thu được một mô hình mạnh hơn.

Đây là một bài toán thuộc lĩnh vực credit risk được sử dụng nhiều trong các tổ chức tài chính, ngân hàng. Chính vì vai trò và mức độ quan trọng của bài toán nên Home Credit thu hút được trên 7000 đội tham dự. Dữ liệu của bài toán thuộc dạng có cấu trúc (SQL) gồm nhiều bảng liên hệ với nhau bởi các primary key và foriegn key theo sơ đồ bên dưới:

![image](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

Các bảng dữ liệu bao quát các khía cạnh của một hồ sơ tín dụng như:

* Thông tin hồ sơ khách hàng hiện tại: Mỗi một hồ sơ được xác định bằng một khóa SK_ID_CURR. Trong thông tin hồ sơ chúng ta sẽ biết được thu nhập, số lượng người phụ thuộc, nghề nghiệp, trình độ giáo dục, ngày sinh, có sở hữu xe hay không,.... và rất nhiều các thông tin khác. Các thông tin này có thể chia thành nhóm nhân khẩu học (demographic) liên quan đến cá nhân vay và thông tin tín dụng liên quan đến tình trạng tín dụng của người vay. Các bảng application_train.csv (sử dụng để huấn luyện) và bảng application_test.csv (sử dụng để test). Về mặt cấu trúc các trường trong bảng application_train.csv và application_test.csv là giống nhau ngoại trừ biến mục tiêu TARGET được giấu ở application_test.csv để đánh giá kết quả mô hình sau cùng.

* Thông tin giao dịch: Các lịch sử giao dịch và hành vi mua sắm qua thẻ tín dụng của người tiêu dùng tại home credit. Bảng POSH_CASH_balance.csv

* Thông tin trả nợ: Lịch sử trả nợ của khách hàng trong quá khứ nếu họ đã có các khoản vay trước đó tại home credit. Bảng installments_payment.csv

* Thông tin hồ sơ các khoản vay trước đây tại home credit. Bảng previous_application.csv

* Thông tin các khoản vay tại tổ chức tài chính khác được lưu trữ tại cục tín dụng. Bảng bureau_balance.csv và bureau.csv.

# 2. Tiền xử lý dữ liệu

Do dữ liệu home credit được chia thành nhiều khía cạnh dữ liệu nhỏ và quản lý tại mỗi bảng riêng biệt nên ta cần thống kê các thông tin của dữ liệu theo key là ID hồ sơ và join các thông tin thu được từ mỗi bảng theo key để thu được bảng tổng hợp làm input cho mô hình. Tại bước này chúng ta cần các kĩ năng về data manipulation trên pandas. Cụ thể bạn đọc có thể tham khảo thêm hướng dẫn xử lý dữ liệu với [pandas](https://www.kaggle.com/phamdinhkhanh/gi-i-thi-u-pandas).

Một điểm cần lưu ý đó là thông tin về các hồ sơ tín dụng thường không đầy đủ. Do đó chúng ta cần kiểm tra tỷ lệ các quan sát bị missing ở mỗi trường. Việc kiểm tra tỷ lệ missing giúp đánh giá mức độ đầy đủ về mặt thông tin mà một trường dữ liệu cung cấp. Khi tỷ lệ missing quá lớn, trường dữ liệu thường không đáng tin cậy trong phân loại mô hình và chúng ta cần phải loại những biến này khỏi mô hình.

## 2.1. Khảo sát dữ liệu

Bên dưới ta sẽ kiểm tra số trường và kích thước mẫu của các tập dữ liệu train và test.

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import os
import seaborn as sns

%matplotlib inline

# Checking for all file
os.listdir('../input')

In [ ]:
# Training data
app_train = pd.read_csv('../input/application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

In [ ]:
# Testing data features
app_test = pd.read_csv('../input/application_test.csv')
print('Testing data shape: ', app_test.shape)
app_test.head()

Biến TARGET của tập test đã được giấu đi để đánh giá mô hình được xây dựng từ tập train. Số lượng quan sát của train lớn gấp khoảng 6 lần train. 

Bên dưới ta sẽ thống kê số lượng và tỷ lệ các class Repaid và Not repaid trong tập train.

In [ ]:
app_train['TARGET'].value_counts().plot.bar()
n_group = app_train['TARGET'].value_counts()
n_group_sum = n_group.sum()

print('Repaid: {}'.format(n_group[0]))
print('Not repaid: {}'.format(n_group[1]))

print('Repaid: {:.2f} {}'.format(n_group[0]/n_group_sum*100, '%'))
print('Not repaid: {:.2f} {}'.format(n_group[1]/n_group_sum*100, '%'))

Mẫu có hiện tượng mất cân bằng khi nhóm thiểu chỉ chiếm 8.07% và nhóm đa số chiếm tới 91.93%. Mẫu mất cân bằng có thể dẫn tới một số tác hại đối với mô hình như:

1. Kết quả dự báo của mô hình chỉ thiên về một class. Thậm chí trong một số trường hợp mô hình dự báo chỉ đưa ra một class duy nhất là class đa số.
2. Dễ dàng ngộ nhận mô hình tốt do Accuracy của mô hình đối với mẫu mất cân bằng thường rất cao. Trong trường hợp này ta cần sử dụng đến các chỉ số thay thế như precision, recall, F1-Score, Kappa, ROC Curve. Về nội dung và ý nghĩa của các chỉ số này các bạn có thể tham khảo tại [đánh giá hệ thống phân lớp](https://machinelearningcoban.com/2017/08/31/evaluation/).

Khi gặp hiện tượng mất cân bằng mẫu chúng ta có thể sử dụng nhiều phương pháp khác nhau để biến đổi mẫu về cân bằng. Một trong những phương pháp đó là:

1.	Thu thập thêm nhiều dữ liệu cho mẫu thiểu số nếu việc thu thập thêm dữ liệu là khả thi trên thực tế.
2.	Sử dụng các phương pháp tái chọn mẫu có lặp lại (resampling) nhằm gia tăng số lượng mẫu thiểu số.
3.	Sử dụng kĩ thuật tạo mẫu tổng hợp cho mẫu thiểu (Synthetic Minority Over-sampling Technique - SMOTE). Thay vì lấy các quan sát lập lại, thuật toán sẽ tạo ra các quan sát tổng hợp dựa trên hai hoặc nhiều quan sát mẫu. Các mẫu được lựa chọn có sự tương đương nhau dựa trên thước đo khoảng cách. Các thuộc tính của quan sát tổng hợp sau đó sẽ được tạo ra bằng cách thêm một đại lượng ngẫu nhiên chênh lệch so với các điểm lân cận.
4.	Thêm các hệ số phạt vào mô hình để gia tăng hàm mất mát nhiều hơn nếu dự báo sai các mẫu thiểu. 

Bên dưới ta sẽ thống kê dữ liệu missing và các kiểu dữ liệu numeric, categorical.

In [ ]:
def summary_missing(dataset):
    n_miss = dataset.isnull().sum()
    n_obs = dataset.shape[0]
    n_miss_per = n_miss/n_obs*100
    n_miss_tbl = pd.concat([n_miss, n_miss_per], axis = 1).sort_values(1, ascending = False).round(1)
    n_miss_tbl = n_miss_tbl[n_miss_tbl[1] != 0]
    print('No fields: ', dataset.shape[0])
    print('No missing fields: ', n_miss_tbl.shape[0])
    n_miss_tbl = n_miss_tbl.rename(columns = {0:'Number mising Value', 1:'Percentage missing Value'})
    return n_miss_tbl

summary_missing(app_train)

In [ ]:
def _tbl_dtype(dataset):
    sum_dtype = pd.DataFrame(dataset.dtypes).sort_values(0).rename(columns = {0:'Data Type'})
    return sum_dtype

table_dtype = _tbl_dtype(app_train)
table_dtype

In [ ]:
table_dtype['Data Type'].value_counts()

In [ ]:
# Các dòng dữ liệu dạng object
app_train.select_dtypes('object').head()

Thống kê số lượng các class trong mỗi nhóm đối với dữ liệu dạng object.

In [ ]:
app_train.select_dtypes('object').apply(pd.Series.nunique)

Vẽ biểu đồ phân phối số lượng các quan sát theo nhóm đối với dữ liệu dạng object. Đối với các biến ORGANIZATION_TYPE, OCCUPATION_TYPE ta sẽ nghiên cứu riêng do số lượng class lớn.

In [ ]:
dtypes_object = table_dtype[table_dtype['Data Type'] == 'object'].index.tolist()
dtypes_object = [col for col in dtypes_object if col not in ['OCCUPATION_TYPE', 'ORGANIZATION_TYPE']]


def _plot_bar_classes(cols):
    app_train[cols].value_counts().plot.bar()

plt.figure(figsize = (20, 15))    
for i in range(1, 15, 1):
    plt.subplot(5, 3, i)
    _plot_bar_classes(dtypes_object[i-1])
    plt.title(dtypes_object[i-1])

Thống kê tỷ lệ Repaid/Not Repaid theo các biến dự báo dạng object.

In [ ]:
def _per_categorical(col):
    tbl_per = pd.pivot_table(app_train[['TARGET', col]], index = ['TARGET'], columns = [col], aggfunc = len)
    per_categorical = (tbl_per.iloc[0, :]/tbl_per.iloc[1, :]).sort_values(ascending = True)
    print(per_categorical)
    print('-------------------------------------\n')
    return per_categorical

for col in dtypes_object:
    _per_categorical(col)

Vẽ biểu đồ tỷ lệ Repaid/Not Repaid theo các biến dự báo dạng object.

In [ ]:
def _plot_per_categorical(col):
    tbl_per = pd.pivot_table(app_train[['TARGET', col]], index = ['TARGET'], columns = [col], aggfunc = len)
    per_categorical = (tbl_per.iloc[0, :]/tbl_per.iloc[1, :]).sort_values(ascending = True)
    per_categorical.plot.bar()
    plt.title(col)
    return per_categorical

plt.figure(figsize = (20, 15))
i = 0
for col in dtypes_object:
    i += 1
    plt.subplot(5, 3, i)
    _plot_per_categorical(col)

Thông qua biểu đồ ta cũng hình dung được một vài biến phân loại có sự khác biệt lớn giữa tỷ lệ Repaid/Non Repaid lớn như NAME_CONTRACT_TYPE, GENDER, FLAG_OWN_REALITY do đó đây là những biến có tác động lớn đến biến mục tiêu.

Kiểm tra riêng cho 2 biến ORGANIZATION_TYPE, OCCUPATION_TYPE.

In [ ]:
plt.figure(figsize = (15, 7))
i = 0
for col in ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']:
    i += 1
    plt.subplot(2, 1, i)
    _plot_per_categorical(col)

## 2.2. Nhóm các đặc trưng theo tỷ lệ Repaid/Not Repaid
Để giảm thiểu số lượng đặc trưng (features) ta có thể nhóm những biến có tỷ lệ Repaid/Not Repaid gần bằng nhau vào một nhóm bởi những nhóm này có đặc tính gần giống nhau trong phân loại hợp đồng.

In [ ]:
for col in ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']:
    _per_categorical(col)

In [ ]:
# Nhóm các giá trị rate gần bằng nhau vào 1 nhóm theo schedule_div.
def _devide_group(col, schedule_div = None, n_groups = 3, *kwargs):
    cols = []
    tbl_per_cat = _per_categorical(col)
    
    if schedule_div is None:
        n_cats = len(tbl_per_cat)
        n_val_incat = int(n_cats/n_groups)
        n_odd = n_cats - n_groups*n_val_incat

        for i in range(n_groups):
            if i == n_groups - 1:
                el = tbl_per_cat[(n_val_incat*i):(n_val_incat*(i+1)+n_odd)].index.tolist()
            else:
                el = tbl_per_cat[(n_val_incat*i):n_val_incat*(i+1)].index.tolist()    
            cols.append(el)
    else:
        idx = 0
        for n_cols in schedule_div:
            el_cols = tbl_per_cat[idx:(idx+n_cols)].index.tolist()
            cols.append(el_cols)
            idx += n_cols
                
    return cols

cols_OCCUPATION_TYPE = _devide_group(col = 'OCCUPATION_TYPE', schedule_div = [1, 7, 9, 1])
cols_OCCUPATION_TYPE

In [ ]:
cols_ORGANIZATION_TYPE = _devide_group(col = 'ORGANIZATION_TYPE')
cols_ORGANIZATION_TYPE

Sau khi đã nhóm các features trong một biến phân loại thành các features tổng hợp chúng ta cần cập nhật lại các trường của bảng dữ liệu theo các features mới.

In [ ]:
def _map_lambda_cats(cols_list, colname, x): 
    cats = list(map(lambda x:colname + '_' + str(x), np.arange(len(cols_list)).tolist()))
    for i in range(len(cols_ORGANIZATION_TYPE)):
        if x in cols_list[i]:
            return cats[i]
        
def _map_cats(cols_list, colname, dataset):                    
    return list(map(lambda x: _map_lambda_cats(cols_list, colname, x), 
                    dataset[colname]))

app_train['ORGANIZATION_TYPE'] = _map_cats(cols_ORGANIZATION_TYPE, 'ORGANIZATION_TYPE', app_train)
pd.Series.unique(app_train['ORGANIZATION_TYPE'])

Áp dụng trên tập test.

In [ ]:
app_test['ORGANIZATION_TYPE'] = _map_cats(cols_ORGANIZATION_TYPE, 'ORGANIZATION_TYPE', app_test)
pd.Series.unique(app_test['ORGANIZATION_TYPE'])

Gán lại các biến này bằng dữ liệu sau khi map.

In [ ]:
app_train['OCCUPATION_TYPE'] = _map_cats(cols_OCCUPATION_TYPE, 'OCCUPATION_TYPE', app_train)
app_test['OCCUPATION_TYPE'] = _map_cats(cols_OCCUPATION_TYPE, 'OCCUPATION_TYPE', app_test)

Sau các xử lý trên, dữ liệu trên app_train và app_test sẽ được map theo các nhóm phân loại mới. Chúng ta có thể kiểm tra lại tỷ lệ Repaid/No Repaid của các nhóm biến mới như bên dưới.

In [ ]:
i = 0
plt.figure(figsize = (16, 8))
for col in ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']:
    i += 1
    plt.subplot(2, 1, i)
    _plot_per_categorical(col)

Sau khi đã nhóm xong các biến category thành những features tổng hợp, chúng ta cần biến đổi các biến này sao cho chúng có thể được sử dụng trong mô hình hồi qui. Có 2 cách biến đổi chính có thể áp dụng đó là one-hot coding và tạo biến thứ bậc. Tạo biến thứ bậc thường được áp dụng khi các nhóm có sự khác biệt về mức độ chẳng hạn như xếp hạng học lực: Giỏi, Khá, Trung bình, Yếu. Trong khi đó one-hot coding sẽ biểu diễn một biến category bởi một vector sparse (các phần tử của vector là 0 hoặc 1) có độ dài bằng số lượng các nhóm sao cho đặc trưng tương ứng với giá trị của biến sẽ được gán là 1 và các đặc trưng còn lại là 0. Chúng ta có thể sử dụng hàm `get_dummies()` trong pandas để tự động biến đổi các biến category trong data frame về dạng one-hot coding.

In [ ]:
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

Kiểm tra kích thước dữ liệu sau biến đổi.

In [ ]:
print('app_train shape: ', app_train.shape)
print('app_test shape: ', app_test.shape)

Số lượng các biến đã tăng lên do biến đổi one-hot coding, tuy nhiên có sự chênh lệch giữa tập train và test. Kiểm tra các biến ở app_train không có trong app_test.

In [ ]:
for fea_name in app_train.columns:
    if fea_name not in app_test.columns:
        print(fea_name)

Những biến này không xuất hiện trong tập test là do có một số biến không xảy ra đầy đủ các khả năng. Chúng ta cần loại bỏ những biến không có trong tập train. Chỉ dữ lại những biến có trong cả app_train và app_test:

In [ ]:
TARGET = app_train['TARGET']

# Lệnh align theo axis = 1 sẽ lấy những trường xuất hiện đồng thời trong app_train và app_test
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
# Sau lệnh align biến TARGET bị mất, do đó ta cần gán lại biến này
app_train['TARGET'] = TARGET

print('app_train shape: ', app_train.shape)
print('app_test shape: ', app_test.shape)

## 2.3. Xử lý outlier

Trong một bộ dữ liệu thường có những quan sát bất thường. Nguyên nhân có thể đến từ dữ liệu bị sai định dạng, quá trình nhập liệu sai, quan sát có tính chất đặc biệt,.... Việc tra soát các dữ liệu bất thường có thể giúp khám phá ra một số tình chất của dữ liệu và đồng thời hiệu chỉnh dữ liệu trong trường hợp nhập liệu sai. Kiếm tra dữ liệu bất thường có thể được thực hiện bằng các thống kê mô tả.

In [ ]:
app_train.head()

In [ ]:
app_train['AMT_INCOME_TOTAL'].describe().plot.box()

Ta nhận thấy biến thu nhập có dấu hiệu bất thường dữ liệu khi hầu hết các khoảng ngũ phân vị (quintile) của biến đều thấp quan gốc 0 ngoại trừ một vài trường hợp cao đặc biệt. Điều này cho thấy có sự chênh lệch trong mức thu nhập của những người đi vay.

In [ ]:
app_train['AMT_INCOME_TOTAL'].describe()

Giá trị cao nhất của thu nhập là khoảng 117 triệu USD trong khi trung bình thu nhập chỉ là 168 nghìn USD. Chúng ta sẽ kiểm tra phân phối của biến TARGET theo biến AMT_INCOME_TOTAL ứng với các trường hợp Repaid và No repaid.

In [ ]:
def _plot_density(colname):
    plt.figure(figsize = (10, 8))
    sns.kdeplot(app_train[colname][app_train['TARGET'] == 0], label = 'Target = 0')
    sns.kdeplot(app_train[colname][app_train['TARGET'] == 1], label = 'Target = 1')
    plt.xlabel(colname)
    plt.ylabel('Density')
    plt.title('Distribution of %s'%colname)

_plot_density('AMT_INCOME_TOTAL')

Sử dụng phương pháp 3 sigma để điều chỉnh lại các giá trị nằm ngoài miền $[\mu - 3\sigma, \mu + 3\sigma]$ về trong miền giá trị đó. Đối với giá trị lớn hơn $\mu+3\sigma$ sẽ được gán bằng $\mu+3\sigma$ và tương tự với giá trị nhỏ hơn $\mu - 3\sigma$. Phương pháp này giúp điều chỉnh các outlier về trong khoảng biến thiên cho phép và làm giảm ảnh hưởng chệch gây ra bởi chúng.

In [ ]:
def _zoom_3sigma(col, dataset, dataset_apl):
    '''
    col: Tên cột dữ liệu
    dataset: Bảng dữ liệu gốc sử dụng để tính khoảng 3 sigma
    dataset_apl: Bảng dữ liệu mới áp dụng khoảng 3 sigma được lấy từ dataset.
    '''
    xs = dataset[col]
    mu = xs.mean()
    sigma = xs.std()
    low =  mu - 3*sigma
#     low =  0 if low < 0 else low
    high = mu + 3*sigma
    
    def _value(x):
        if x < low: return low
        elif x > high: return high
        else: return x
    xapl = dataset_apl[col]    
    xnew = list(map(lambda x: _value(x), xapl))
    n_low = len([i for i in xnew if i == low])
    n_high = len([i for i in xnew if i == high])
    n = len(xapl)
    print('Percentage of low: {:.2f}{}'.format(100*n_low/n, '%'))
    print('Percentage of high: {:.2f}{}'.format(100*n_high/n, '%'))
    print('Low value: {:.2f}'.format(low))
    print('High value: {:.2f}'.format(high))
    return xnew

# Kiểm tra với biến FLAG_MOBIL
x = _zoom_3sigma('FLAG_MOBIL', app_train, app_train)    

In [ ]:
app_train.dtypes.unique()

In [ ]:
# Thống kê các giá trị khác biệt trong toàn bộ các biến.
def _count_unique(x):
    return pd.Series.nunique(x)

tbl_dis_val = app_train.apply(_count_unique).sort_values(ascending = False)
tbl_dis_val[tbl_dis_val > 500]

Ta coi các biến có số lượng giá trị khác biệt > 500 là các biến liên tục. Áp dụng nguyên lý 3 sigma cho các biến này.

In [ ]:
cols_3sigma = tbl_dis_val[tbl_dis_val > 500].index.tolist()
# Loại bỏ biến key là SK_ID_CURR ra khỏi danh sách:
cols_3sigma = cols_3sigma[1:]

In [ ]:
# Loại bỏ các outlier bằng 3 sigma
for col in cols_3sigma:
    print(col)
    app_train[col] = _zoom_3sigma(col, app_train, app_train) 
    print('------------------------\n')

In [ ]:
for col in cols_3sigma:
    print(col)
    app_test[col] = _zoom_3sigma(col, app_train, app_test) 
    print('------------------------\n')

In [ ]:
# Kiểm tra lại biến AMT_INCOME_TOTAL sau khi loại bỏ outlier
app_train['AMT_INCOME_TOTAL'].describe().plot.box()

In [ ]:
_plot_density('AMT_INCOME_TOTAL')

Như vậy sau khi loại bỏ oulier, đồ thị phân phối của AMT_INCOME_TOTAL đã không còn thiên lệch. Các điểm mốc Q1, Q2, Q3, Q4 trong tứ phân vị (quintile) đã không quá nhỏ so với giá trị lớn nhất trong biểu đồ box plot như ban đầu. Biểu đồ hàm mật độ cũng cho thấy một phân phối không có các giá trị dị biệt như trước khi điều chỉnh.

## 2.4. Xử lý dữ liệu missing

Xử lý dữ liệu missing là một trong những mảng rất quan trọng của tiền xử lý dữ liệu (pre-processing data). Các phương pháp xử lý dữ liệu missing cũng đa dạng, từ đơn giản đến phức tạp. Những phương pháp thông thường nhất đó là thay thế các điểm dữ liệu bị missing bằng các giá trị đại diện chẳng hạn như median, mean, hoặc giá trị có tần xuất xuất hiện lớn nhất (mode). Một số phương pháp phức tạp hơn sẽ dựa trên khoảng cách của các điểm dữ liệu để thay các dữ liệu missing bằng giá trị của quan sát gần nhất với nó hoặc thực hiện voting dữ liệu trong một tập hợp gần nhất bằng trung bình có trọng số hoặc không có trọng số của các điểm này. 
Trong phân tích này tôi sẽ áp dụng MinMaxScaler để chuẩn hóa biến dự báo và sử dụng mean để xử lý dữ liệu missing.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, Imputer

if 'TARGET' in app_train.columns:
    TARGET = app_train.pop('TARGET')

# Gán train và test vào app_train và app_test; train, test được sử dụng để scale dữ liệu
train = app_train
test = app_test

# Khởi tạo inputer theo phương pháp trung bình
inputer = Imputer(strategy = 'mean')
inputer.fit(train)

# Điền các giá trị NA bằng trung bình
train = inputer.transform(train)
test = inputer.transform(test)

# Khởi tạo scaler theo phương pháp MinMaxScaler trong khoảng [-1, 1]
scaler = MinMaxScaler(feature_range = (-1, 1))
scaler.fit(train)

# Scale dữ liệu trên train và test
train = scaler.transform(train)
test = scaler.transform(test)

# Loại bỏ cột SK_ID_CURR đầu tiên do cột này là key. Khi cần lấy từ app_train và app_test sang
train = train[:, 1:]
test = test[:, 1:]

print('train shape: ', train.shape)
print('test shape: ', test.shape)

# 3. Xây dựng mô hình
## 3.1. Đánh giá mức độ tương quan

Trong một mô hình có quá nhiều biến input, chúng ta cần loại bỏ các biến không quan trọng để giảm chi phí tính toán và hạn chế khả năng overfiting của mô hình. Một cách phổ biến nhất trong đánh giá mối quan hệ giữa biến dự báo và biến mục tiêu đó là sử dụng bảng hệ số tương quan. Có một vài phương pháp tiếp cận khác cao cấp hơn như:

* Thông qua các mô hình ensemble để xếp hạng mức độ quan trọng của các biến trong tree boosting hoặc random forest.
* Sử dụng chỉ số Information Value trong Scorecard. Cách này được sử dụng phổ biến trong các mô hình credit risk.
* Sử dụng chỉ số Akaike Inforamtion Criterion kết hợp với phương pháp step wise áp dụng trong các mô hình thuộc lớp hồi qui.

Có thời gian mình sẽ trình bày thêm về các phương pháp này. Tạm thời chúng ta đánh giá quan hệ giữa các biến thông qua mức độ tương quan.

In [ ]:
app_train['TARGET'] = TARGET
corr_tbl = app_train.corr()
corr_tbl

In [ ]:
corr_tbl['TARGET'].sort_values()

Chúng ta có thể sử dụng hàm np.corrcoef(arr) để tính ma trận hệ số tương quan cho các biến. Tuy nhiên tính toán correlation trên numpy cần nhiều tài nguyên hơn so với pandas. Với lệnh trên, máy của mình đã bung memory. Một lưu ý nhỏ đó là các chiều dữ liệu được coi như các dòng và các quan sát là các cột trong lệnh np.corrcoef(arr). Chuyển ma trận qua data frame và tính correlation.

In [ ]:
pd_train = pd.DataFrame(train, columns = app_train.columns[1:-1])
pd_train['TARGET'] = TARGET
pd_train.head()

In [ ]:
corr_tbl_train = pd_train.corr()
corr_tbl_train

Mức độ tương quan của các biến sẽ được xếp hạng một cách tương đối dựa trên giá trị tuyệt đối của chúng. Các khoảng đánh giá như sau:

* 0-0.19: Rất yếu.
* 0.2-0.39: Yếu.
* 0.4-0.59: Trung bình.
* 0.6-0.79: Cao.
* 0.8-1: Rất cao.

Dựa trên bảng hệ số tương quan chúng ta có thể tìm ra những biến có mối liên hệ lớn tới biến TARGET.

In [ ]:
corr_tbl_train['TARGET'].sort_values()

Chọn 15 biến có mức độ tương quan lớn nhất đến biến mục tiêu và biểu diễn ma trận hệ số tương quan của chúng.

In [ ]:
# Lấy ra danh sách 15 biến có tương quan lớn nhất tới biến mục tiêu theo trị tuyệt đối.
cols_corr_15 = np.abs(corr_tbl_train['TARGET']).sort_values()[-16:].index.tolist()

# Tính ma trận hệ số tương quan
cols_tbl_15 = pd_train[cols_corr_15].corr()

# Biểu diễn trên biểu đồ heatmap
plt.figure(figsize = (13, 10))
sns.heatmap(cols_tbl_15, cmap = plt.cm.RdYlBu_r, annot = True)

Phân phối xác xuất của các biến trên theo các nhóm của biến mục tiêu.

In [ ]:
plt.figure(figsize = (20, 5))
for i in range(5):
    _plot_density(cols_corr_15[i])

DAYS_BIRTH là một biến liên tục có range nằm trong khoảng từ 20 - 70. Trục hoành biểu diễn độ tuổi theo ngày, giá trị âm thể hiện số ngày chênh lệch từ thời điểm sinh so với hiện tại. Như vậy người trẻ sẽ nằm ở bên phải của trục hoành và người già sẽ nằm bên trái. Từ biểu đồ ta có thể thấy những người trẻ có xu hướng không trả nợ nhiều hơn những người già. Bằng chứng là tại đồ thị mật độ ta nhận thấy phân phối lệch phải. Để làm rõ hơn nhận định, chúng ta đi tìm hiểu hành vi trả nợ theo các nhóm tuổi.

In [ ]:
age_bin = app_train[['TARGET', 'DAYS_BIRTH']]
age_bin['YEAR_OLD'] = -app_train['DAYS_BIRTH']/365

# Phân chia khoảng tuổi thanh 10 khoảng bằng nhau
age_bin['DAYS_BIN'] = pd.cut(age_bin['YEAR_OLD'], bins = np.linspace(20, 70, num = 11))
age_bin.head()

Thống kê số lượng không trả nợ theo các khoảng tuổi.

In [ ]:
age_bin.groupby(['DAYS_BIN']).mean()

Biểu đồ tỷ lệ trả nợ các hợp đồng theo từng nhóm tuổi.

In [ ]:
plt.figure(figsize = (8, 6))
age_bin.groupby(['DAYS_BIN']).mean()['TARGET'].plot.barh(color = 'b')
plt.xticks(rotation = '75')
plt.xlabel('Not Repaid rate')

Biểu đồ cho thấy tỷ lệ không trả nợ ở những người thuộc nhóm tuổi cao thấp hơn so với nhóm tuổi thấp. Hành vi này có thể là do người trẻ từ 20-25 tuổi chưa có thu nhập cao và các khoản tiết kiệm trong khi người gia có nhiều tiền tiết kiệm. 

Sau khi xử lý dữ liệu missing và tìm hiểu phân phối tỷ lệ Repaid/ Not Repaid của các biến numeric. Chúng ta sẽ hồi qui mô hình.

## 3.2. Logistic regression

Mặc dù tên mô hình là logistic regression nhưng logistic không phải là mô hình hồi qui mà trái lại, là mô hình thuộc lớp phân loại. Ngoài ra logistic không phải là một phương pháp mạnh. Đây là lớp mô hình có đường biên phân loại tuyến tính (linear seperable) nên khả năng phân loại dữ liệu trong trường hợp biên của các nhóm chồng lấn sẽ yếu. Tuy nhiên đây là phương pháp được sử dụng phổ biến nhất trong tính toán scorecard. Một trong những nguyên nhân giúp logistic được ưa chuộng trong scorecard đó là:

* Giá trị của logistic nằm giới hạn trong khoảng [0, 1] và phù hợp với miền giá trị mà một xác xuất rơi vào.
* Đạo hàm của hàm sigmoid rất đẹp mắt..
* Hàm mất mát là một hàm lồi.
* Có thể giải thích được tác động của các biến giải thích lên biến mục tiêu dựa vào hệ số ước lượng. Điều mà trường phái thống kê rất coi trọng.

Chính vì thế, khi làm quen với Machine Learning, hồi qui logistic sẽ là một trong những lớp bài toán được tiếp cận đầu tiên sau hồi qui tuyến tính. Bên dưới là mô hình hồi qui logistic áp dụng cho bài toán này.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Xây dựng mô hình logistic với tham số kiểm soát C = 0.0001
log_reg = LogisticRegression(C = 0.0001)

# Huấn luyện mô hình
log_reg.fit(train, TARGET)

In [ ]:
train_pred_prob = log_reg.predict_proba(train)[:, 1]

In [ ]:
TARGET.value_counts()/TARGET.value_counts().sum()

ROC curve (Receiver Operating Characteristic) là một đường cong thể hiện mối liên hệ giữa tỷ lệ mắc sai lầm loại I (false positive rate) và tỷ lệ dự báo đúng positive (true positive rate). 

Trong thống kê, chúng ta chia các sai lầm của giả thuyết thành 2 loại: sai lầm loại I và sai lầm loại II. Khi thực hiện một mô hình phân loại nợ mục tiêu của chúng ta là tìm ra những hồ sơ nợ xấu. Một kết luận từ mô hình có thể rơi vào 2 trạng thái sai lầm: nhận định một hồ sơ xấu là tốt - loại I, hoặc trái lại, coi một hồ sơ tốt là xấu - loại II (giết nhầm còn hơn bỏ sót). 

Tác hại của sai lầm loại I sẽ lớn hơn sai lầm loại II bởi ảnh hưởng do hồ sơ xấu gây là lớn hơn nhiều so với việc bạn kiếm được một hồ sơ tốt. Đó cũng là lý do Tào Tháo thường hay đa nghi bởi ông biết được mức độ tác hại của 2 loại sai lầm này là khác nhau (mặc dù ông cũng chẳng có một chứng chỉ thống kê nào).

Nếu chúng ta chấp nhận một tỷ dự báo đúng hồ sơ tốt cao hơn thì chúng ta sẽ phải hạ thấp ngưỡng threshold xác định loại hồ sơ (mặc định là 0.5). Điều này dẫn đến các hồ sơ xấu có khả năng bị nhận định là hồ sơ tốt cao hơn. Điều này cho thấy luôn có sự đánh đổi giữa tỷ lệ true positive rate và false positive rate. 

Một mô hình phân loại tốt là mô hình mà ở các threshold ta phân loại được nhiều nhất các hồ sơ tốt nhưng chỉ phải chấp nhận một lượng rất nhỏ các hồ sơ xấu. Các mô hình như vậy đều có chung một tính chất, đó là đường cong ROC lồi lên phía trên so với trục hoành. Như vậy các bạn đã hiểu được ý nghĩa của đường cong ROC rồi chứ? Trên thực tiễn ROC là một biểu đồ trực quan đánh giá phẩm chất của mô hình. ROC càng lồi mô hình càng phân loại tốt và trái lại. Mức độ lồi của đường cong ROC được đo bằng phần diện tích nằm dưới đường cong ROC và trục hoành hay còn gọi là chỉ số AUC (Area under curve).

Trong tín dụng sau khi đã tìm được mô hình sở hữu đường cong ROC tốt nhất, ngân hàng sẽ xây dựng một hàm mất mát dựa trên phương trình ước lượng tổn thất:
$$EL = PD*LGD$$
Trong đó EL - Expected Loss, PD - Probability Default, LGD - Loss given default.

Để chọn ra một threshold tối ưu mang lại tổn thất về mặt lợi ích cho ngân hàng là nhỏ nhất. Về EL là gì? PD, LGD là gì? Mình sẽ không giải thích thêm ở bài viết này. Các bạn muốn đi sâu vào ngành Credit Risk có thể tìm thấy trong rất nhiều các tài liệu và khóa học về mảng đề tài này.

Bên dưới ta sẽ xây dựng một đường cong ROC.

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve
fpr, tpr, thres = precision_recall_curve(TARGET, train_pred_prob)

def _plot_roc_curve(fpr, tpr, thres):
    plt.figure(figsize = (10, 8))
    plt.plot(fpr, tpr, 'b-', label = 'ROC')
    plt.plot([0, 1], [0, 1], '--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')

_plot_roc_curve(fpr, tpr, thres)

Trong các mô hình phân loại chúng ta thường quan tâm đến tỷ lệ Accuracy. Đây là chỉ số đại diện nhất vì nó phản ánh trực tiếp hiệu quả của một mô hình thông qua số trường hợp phân loại đúng /tổng số trường hợp phân loại. Tuy nhiên chỉ số này tỏ ra kém hiệu quả đối với trường hợp mẫu mất cân bằng nghiêm trọng. Trong trường hợp này kết quả dự báo của mô hình thường thiên lệch về một lớp đa số. Do đó Accuracy hiển nhiên cao đối với những mô hình phẩm chất kém bởi số trường hợp hồ sơ tốt là quá nhiều dẫn đến một một mô hình cảm tính quyết định toàn bộ là hồ sơ tốt cũng dẫn tới Accuracy lớn. Một modeler thiếu kinh nghiệm sẽ rất dễ hài lòng với kết quả này. Nếu đưa vào áp dụng những mô hình như vậy sẽ ảnh hưởng không nhỏ tới các tổ chức tài chính, kinh doanh. Vậy trong trường hợp này tỷ lệ nào được sử dụng thay thể để đánh giá mô hình?
Precision và Recall là một lựa chọn thay thế phù hợp nhất bởi nó thỏa mãn các tính chất:

* Cả 2 đều đánh mức độ dự báo chính xác của positive tức hồ sơ phân loại là xấu. Đây là nhóm được ưu tiên phân loại chính xác hơn vì thiệt hại gây ra bởi nó lớn hơn.
* Precision đánh giá tỷ lệ dự báo chính xác hồ sơ xấu trong tổng số trường hợp được dự báo là xấu.
* Recall đánh giá tỷ lệ dự báo chính xác hồ sơ xấu khi hồ sơ về mặt bản chất là xấu.

Bên cạnh đó còn có các chỉ số khác chúng ta có thể cân nhắc như F1-Score, Kappa, Gini.

Khi nhìn vào biểu đồ của Precision và Recall ta sẽ biết được tại mỗi một ngưỡng threshold sẽ trả về tỷ lệ phân loại hồ sơ xấu là bao nhiêu trên lần lượt tổng số các hồ sơ được dự báo là xấu và tổng số các hồ sơ xấu trên thực tế.

In [ ]:
from sklearn.metrics import precision_recall_curve
prec, rec, thres = precision_recall_curve(TARGET, train_pred_prob)

def _plot_prec_rec_curve(prec, rec, thres):
    plt.figure(figsize = (10, 8))
    plt.plot(thres, prec[:-1], 'b--', label = 'Precision')
    plt.plot(thres, rec[:-1], 'g-', label = 'Recall')
    plt.xlabel('Threshold')
    plt.ylabel('Probability')
    plt.title('Precsion vs Recall Curve')
    plt.legend()

_plot_prec_rec_curve(prec, rec, thres)

Đường ROC curve không lồi lên phía trên so với trục hoành cho thấy sức mạnh phân loại của mô hình tương đối yếu. Tỷ lệ precision và recall theo các ngưỡng threshold cũng không phải là các đường cong lồi dẫn đến khi thay đổi threshold có thể làm tăng chi phí đánh đổi giữa tỷ lệ precision và recall đáng kể. Ở mỗi mức threshold không đạt được đồng thời tỷ lệ cao ở precision và recall. Thay vào đó ta sẽ phải đánh đổi giữa precision cao hoặc recall cao. Đây không phải là một mô hình đủ tốt để áp dụng vào thực tiễn.

## 3.3. Sử dụng kĩ thuật feature engineering

Hầu hết bí quyết để chiến thắng trong các cuộc thi phân tích dữ liệu đó là feature engineering. Feature engineering quan trọng đến mức thầy Andrew Ng đã nói rằng: 'Áp dụng học máy cơ bản là sử dụng feature engineering'. Và theo kinh nghiệm của mình, feature engineering thường mang lại kết quả tốt hơn so với các phương pháp hyparameter tunning. Tất nhiên sử dụng kết hợp cả 2 phương pháp này là tốt nhất nhưng mình vẫn thường có xu hướng thực hiện feature engineering trước khi lựa chọn các phương pháp phức tạp. Tránh tình trạng garbage in garbage out. Một mô hình phức tạp nhưng không có các biến đầu vào tin cậy thì kết quả mô hình sẽ không thể tốt.

Với mô hình hiện tại chúng ta nhận thấy các biến không đủ mạnh để tạo ra một kết quả chuẩn xác. Trong trường hợp này mình nghĩ ngay đến việc áp dụng feature engineering như một giải pháp cải thiện tình hình. feature engineering sẽ tạo ra những biến mới mà khả năng giải thích của nó sẽ tốt hơn đáng kể nếu áp dụng vào mô hình. 

Về kĩ thuật feature engineering chúng ta có 2 nhánh chính là:

* feature selection: lựa chọn các biến tốt nhất từ các biến đã có. Phương pháp này có thể dựa trên ranking của các feature theo các tiêu chí đánh giá sức mạnh hoặc dựa trên kinh nghiệm và hiểu biết về đề tài nghiên cứu.

* thêm mới: Tạo ra các feature mới từ các feature sẵn có. Các biến đổi có thể là scale, chuẩn hóa dữ liệu, sử dụng lũy thừa, logarith,....

Trong bài này mình sẽ sử dụng feature engineering theo phương pháp thêm mới bằng đa thức bậc cao (polynormal feature). Các biến được lựa chọn để feature chỉ bao gồm 15 biến có tương quan cao nhất. Bậc được lựa chọn cao nhất là 3. Thông qua polynormal feature các biến bậc cao và biến tích chéo sẽ được tạo thành. Chẳng hạn từ 2 biến EXT_SOURCE_1 và EXT_SOURCE_2 chúng ta có thể tạo ra EXT_SOURCE_1^2, EXT_SOURCE_2^2 và EXT_SOURCE_1xEXT_SOURCE_2.

Trong sklearn chúng ta có thể dễ dàng sử dụng kĩ thuật này thông qua hàm `PolynormialFeatures()`.

In [ ]:
print(cols_corr_15)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, Imputer, MinMaxScaler

# Khởi tạo các preprocessing. Trong đó inputer theo mean, minmax scaler theo khoảng 0, 1 và polynomial features bậc 3.
inputer = Imputer(strategy = 'mean')
minmax_scaler = MinMaxScaler(feature_range = (0, 1))
poly_engineer = PolynomialFeatures(degree = 3)

# Lấy các feature có tương quan lớn nhất đến biến mục tiêu từ app_train và app_test
TARGET = app_train[cols_corr_15[-1]]
train_poly_fea = app_train[cols_corr_15[:-1]]
test_poly_fea = app_test[cols_corr_15[:-1]]

# input dữ liệu missing
inputer = inputer.fit(train_poly_fea)
train_poly_fea = inputer.transform(train_poly_fea)
test_poly_fea = inputer.transform(test_poly_fea)

# Minmax scaler dữ liệu
minmax_scaler = minmax_scaler.fit(train_poly_fea)
train_poly_fea = minmax_scaler.transform(train_poly_fea)
test_poly_fea = minmax_scaler.transform(test_poly_fea)

print('train_poly_fea shape: ', train_poly_fea.shape)
print('test_poly_fea shape: ', test_poly_fea.shape)

In [ ]:
# Polynormial features dữ liệu
poly_engineer = poly_engineer.fit(train_poly_fea)
train_poly_fea = poly_engineer.transform(train_poly_fea)
test_poly_fea = poly_engineer.transform(test_poly_fea)

print('train_poly_fea shape: ', train_poly_fea.shape)
print('test_poly_fea shape: ', test_poly_fea.shape)

Sau feature engineering, số lượng các biến đã tăng từ 15 lên 816 biến. Chúng ta có thể xem danh sách các features được tạo mới như sau:

In [ ]:
features = poly_engineer.get_feature_names(input_features = cols_corr_15[:-1])
features[:10]

Để đánh giá liệu rằng sau khi thực hiện features engineering có giúp cải thiện kết quả hay không chúng ta thực hiện hồi qui logistic theo những features mới.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Xây dựng mô hình hồi qui logistic với tham số kiểm soát là C = 0.0001
lg_reg = LogisticRegression(C = 0.0001)
lg_reg.fit(train_poly_fea, TARGET)
lg_reg

In [ ]:
# Dự báo xác xuất logistic
train_pred_prob = lg_reg.predict_proba(train_poly_fea)[:, 1]

In [ ]:
# Biểu diễn đường roc_curve
from sklearn.metrics import roc_curve
fpr, tpr, thres = roc_curve(TARGET, train_pred_prob)


def _plot_roc_curve(fpr, tpr, thres):
    roc = plt.figure(figsize = (10, 8))
    plt.plot(fpr, tpr, 'b-', label = 'ROC')
    plt.plot([0, 1], [0, 1], '--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    return roc

# Lưu biểu đồ vào p1
p1 = _plot_roc_curve(fpr, tpr, thres)

In [ ]:
from sklearn.metrics import auc
#0.7127599620726505
auc(fpr, tpr)

So với trước khi thực hiện feature engineering đường cong ROC sau khi thực hiện feature engineering lồi hơn. Điều đó cho thấy sức mạnh phân loại của mô hình tốt hơn sau feature engineering.

In [ ]:
from sklearn.metrics import precision_recall_curve, accuracy_score

prec, rec, thres = precision_recall_curve(TARGET, train_pred_prob)

def _plot_prec_rec_curve(prec, rec, thres):
    plot_pr = plt.figure(figsize = (10, 8))
    plt.plot(thres, prec[:-1], 'b--', label = 'Precision')
    plt.plot(thres, rec[:-1], 'g-', label = 'Recall')
    plt.xlabel('Threshold')
    plt.ylabel('Probability')
    plt.title('Precsion vs Recall Curve')
    return plot_pr

_plot_prec_rec_curve(prec, rec, thres)

In [ ]:
# Accuracy
train_pred_label = lg_reg.predict(train_poly_fea)
accuracy_score(TARGET, train_pred_label)

## 3.4. Random Forest

Random Forest là mô hình thuộc lớp kết hợp (ensemble model) tức kết quả được đưa ra dựa trên không chỉ một mô hình mà từ nhiều mô hình khác nhau. 

Random Forest sẽ xây dựng một rừng cây ngẫu nhiên dựa trên các node và nhánh. Đại diện cho mỗi node là một câu hỏi mà giá trị trả về là YES hoặc NO. Các nhánh sẽ có tác dụng kết nối các nodes để tạo ra một kịch bản đường đi (routine).

Node bắt đầu của Random Forest là root node. Từ root node, mô hình sẽ xây dựng một bộ câu hỏi Yes/No dựa trên thông tin được cung cấp từ biến dự báo. Các nhánh YES, NO sẽ rẽ đến các node mới được gọi là internal node. Chẳng hạn đối với biến liên lục như YEAR_OLD mô hình có thể đặt ra câu hỏi YEAR_OLD > 30 hay không? Dựa trên giá trị của quan sát đối với biến YEAR_OLD mà một quan sát có thể rẽ theo nhánh YES hoặc NO. 

Tại phía cuối của các nhánh YES/NO mô hình tiếp tục khởi tạo những internal node ở tầng thấp hơn với các biến khác. Thứ tự các biến được lựa chọn là ngẫu nhiên. Quá trình rẽ nhánh được thực hiện liên tục cho đến khi mô hình đi đến node cuối. Tại node này không có nhánh nào được rẽ thêm. 

Sơ đồ của mô hình rất giống một cái cây. Xuất phát từ gốc cây rẽ vào các cành to, cành nhỏ và kết thúc ở lá. Bởi thế node cuối cùng còn được gọi là leaf node. Tại leaf node mô hình sẽ đưa ra kết quả thống kê của kịch bản đường đi (routine) từ gốc tới lá là một giá trị xác xuất của positive và negative. Mỗi một kịch bản rẽ nhánh từ root node tới leaf node được gọi là một cây.

Lưu ý rằng mỗi một cây được sẽ được áp dụng trên nhiều mẫu dữ liệu con được lựa chọn ngẫu nhiên để quyết định nhãn cho biến mục tiêu tại leaf node. Một quan sát được dự báo trên rất nhiều cây khác nhau và kết quả nhãn trả về từ các cây sẽ là cơ sở để voting nhãn cho quan sát.

Kết quả từ mô hình Random Forest được kết hợp từ nhiều cây quyết định con và được thử nghiệm trên nhiều bộ dữ liệu con nên sai số dự báo thông thường nhỏ hơn so với những mô hình phân loại tuyến tính như logistic hoặc linear regression. 

Bên cạnh Random Forest thì Gradient Boosting và AdaBoost cũng là các mô hình thuộc lớp mô hình kết hợp thường được áp dụng và mang lại hiệu quả bất ngờ tại nhiều cuộc thi.

Để sử dụng Random Forest trong python chúng ta có thể khai thác module `sklearn.ensemble`

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Khởi tạo rừng cây
rd_classifier = RandomForestClassifier(n_estimators = 100, # Số cây trong rừng cây
                                       max_depth = 5, # Độ sâu của cây
                                       random_state = 123, # Khai báo seed để mô hình không đổi cho các lần chạy sau
                                       verbose = 1, # In log của quá trình huấn luyện
                                       n_jobs = -1 # Sử dụng đa luồng để vận hành mô hình
                                      )
rd_classifier

In [ ]:
# Huấn luyện mô hình
rd_classifier.fit(train_poly_fea, TARGET)

# Dự báo trên tập train
train_prob_rd = rd_classifier.predict_proba(train_poly_fea)[:,1]

Vẽ biểu đồ đường ROC curve

In [ ]:
fpr2, tpr2, thres2 = roc_curve(TARGET, train_prob_rd)
p2 = _plot_roc_curve(fpr, tpr, thres)

Biểu diễn cả 2 đường ROC của hồi qui logistic và random forest trên cùng một đồ thị.

In [ ]:
plt.figure(figsize = (10, 8))
plt.plot(fpr2, tpr2, 'b-', label = 'Random Forest')
plt.plot(fpr, tpr, 'r-', label = 'Logistic')
plt.plot([0, 1], [0, 1], '--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()

In [ ]:
from sklearn.metrics import auc
#0.7300858815170105
auc(fpr2, tpr2)

Ta nhận thấy đường ROC của Random Forest lồi hơn so với Logistic. Điều đó cho thấy sức mạnh phân loại của Random Forest là tốt hơn Logistic.

Biểu đồ precision và recall curve

In [ ]:
prec, rec, thres = precision_recall_curve(TARGET, train_pred_prob)
_plot_prec_rec_curve(prec, rec, thres)

Một trong những ưu thế của mô hình random forest đó là khả năng đánh giá mức độ quan trọng của các biến dự báo dựa trên mức độ ảnh hưởng của nó tới xác xuất của biến mục tiêu. Đây là một trong những phương pháp feature selection thường được sử dụng trong machine learning để tuyển chọn biến. Thông tin về mức độ quan trọng có thể được khai thác thông qua thuộc tính `feature_importances_` như sau:

In [ ]:
# Lấy thông tin về mức độ quan trọng các biến tác động lên biến mục tiêu
feature_importance = rd_classifier.feature_importances_
feature_importance = pd.DataFrame({'importance values': feature_importance})
feature_importance.index = features
feature_importance = feature_importance.sort_values('importance values', ascending = False)
feature_importance[:10]

In [ ]:
feature_importance[:10].sort_values('importance values', ascending = True).plot.barh(figsize = (8, 6))
plt.yticks(rotation = 15)
plt.xlabel('Importance values')

In [ ]:
feature_importance.iloc[:5, 0].tolist()

Tính toán mức độ chính xác của mô hình dự báo.

In [ ]:
from sklearn.metrics import accuracy_score
train_label_rd = rd_classifier.predict(train_poly_fea)
accuracy_score(train_label_rd, TARGET)

In [ ]:
np.unique(train_label_rd, return_counts = True)

## 3.5. Gradient Boosting

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import gc

In [ ]:
lgb_classifier = lgb.LGBMClassifier(n_estimator = 10000, 
                                    objective = 'binary', 
                                    class_weight = 'balanced',
                                    learning_rate = 0.05,
                                    reg_alpha = 0.1,
                                    reg_lambda = 0.1,
                                    subsample = 0.8,
                                    n_job = -1,
                                    random_state = 12
                                   )
lgb_classifier

In [ ]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = 12)
valid_scores = []
train_scores = []
count = 0
for train_idx, valid_idx in kfold.split(train_poly_fea):
    count += 1
    # Split train, valid
    train_features, train_labels = train_poly_fea[train_idx], TARGET[train_idx]
    valid_features, valid_labels = train_poly_fea[valid_idx], TARGET[valid_idx]
    lgb_classifier.fit(train_features, train_labels, eval_metric = 'auc',
              eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
              eval_names = ['valid', 'train'], 
              early_stopping_rounds = 100, verbose = 200)
    
    valid_score = lgb_classifier.best_score_['valid']['auc'] 
    train_score = lgb_classifier.best_score_['train']['auc'] 
    
    valid_scores.append(valid_score)
    train_scores.append(train_score)
    
    print('fold time: {}; train score: {}; valid score: {}'.format(count, valid_score, train_score))

## 3.6. Neural network

In [ ]:
# Deep learning với Keras
from keras.layers import Input, Dense, Flatten, Concatenate, concatenate, Dropout, Lambda
from keras.models import Model
from keras.layers.embeddings import Embedding

In [ ]:
train.shape

In [ ]:
# design neural network
input_els = []
encode_els = []

# Generate a list include many Input layers

for i in range(train.shape[1]):
    # input alway have the shape (*, 1)
    input_els.append(Input(shape = (1,)))
    encode_els.append(input_els[-1])
# encode_els

In [ ]:
# concate nate all layers
encode_els = concatenate(encode_els) 

# After completed the input layers, we design the hidden layers
hidden1 = Dense(units = 128, kernel_initializer = 'normal', activation = 'relu')(encode_els)
droplayer1 = Dropout(0.2)(hidden1)
hidden2 = Dense(64, kernel_initializer = 'normal', activation = 'relu')(droplayer1)
droplayer2 = Dropout(0.2)(hidden2)
outputlayer = Dense(1, kernel_initializer = 'normal', activation = 'sigmoid')(droplayer2)

classifier = Model(input = input_els, outputs = [outputlayer])

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.summary()

In [ ]:
# split train/valid
from sklearn.model_selection import KFold
count = 0
kfold = KFold(n_splits = 10, shuffle = True, random_state = 12)
valid_scores = []
train_scores = []
for train_idx, valid_idx in kfold.split(train_poly_fea):
    while count < 1:
        count += 1
        # Split train, valid
        train_features, train_labels = train[train_idx], TARGET[train_idx]
        valid_features, valid_labels = train[valid_idx], TARGET[valid_idx]
        classifier.fit(
            [train_features[:, i] for i in range(train.shape[1])], #lấy list toàn bộ các cột
            train_labels,
            epochs=1,
            batch_size=128,
            shuffle=True,
            validation_data=([valid_features[:, i] for i in range(train.shape[1])], valid_labels) 
        )

In [ ]:
# DỰ báo trên tập train.
train_prob_nn = classifier.predict([train[:, i] for i in range(train.shape[1])])
train_prob_nn

In [ ]:
# np.save('train_prob_nn.npy',train_prob_nn)

fpr4, tpr4, thres4 = roc_curve(TARGET, train_prob_nn)
_plot_roc_curve(fpr4, tpr4, thres4)

In [ ]:
from sklearn.metrics import auc
auc(fpr4, tpr4)

In [ ]:
prec, rec, thres = precision_recall_curve(TARGET, train_prob_nn)
_plot_prec_rec_curve(prec, rec, thres)

In [ ]:
# Save data
# np.save('train1.npy', train)
# np.save('test1.npy', test)
# np.save('TARGET.npy', TARGET)


In [ ]:
# import numpy as np
# train = np.load('train.npy')
# print(train.shape)
# test = np.load('test.npy')
# print(test.shape)